# Data input and cleaning
Read data from 'ods' or '.xlsx' file

## Data format
### Output
The final cleaned data has 3 levels of indexes and 4 columns ('value','normalised': data normalised according to expected upper and lower bounds, 'UPPER': upper bound and 'LOWER':lower bound). The 1st level of index has no purpose (it could serve as another level of toggling since it's a higher level of categorisation for the next indexing layer). The last index is the date.

For reference, the final dataset looks like this

```
                                        Value       Normalised value       Upper limit       Lower limit

Category       Subcategory       Date
```

### Input
The initial data that is used here is a medical record file containing time series data file, in row format. This notebook is dedicated to cleaning the data. 

For reference, the data was in this format
```
                                 Upper Limit       Lower Limit     Date 1       Date 2       Date 3

Category A     Subcategory 1       <ulA1>       <llA1>       <val>       <val>

..

       <null row>

Category B      Subcateory 1       <ulB1>       <llB1>       <val>       <val>

                Subcategory 2       <ulB2>       <llB2>       <val>       <val>

..

.....
```

In [1]:
import pandas as pd
import numpy as np

Specify the columns

In [2]:
UPPER = 'UPPER'
LOWER = 'LOWER'

### Read raw data from file
Reports data

In [3]:
FILENAME = 'values.ods'

In [4]:
import odf # For supporting '.ods' filetypes
if '.ods' in FILENAME:
    f = pd.read_excel(FILENAME, engine='odf')
elif '.xlsx' in FILENAME:
    f = pd.read_excel(FILENAME) 
elif '.csv' in FILENAME:
    f = pd.read_csv(FILENAME)
else:
    raise ValueError("Unknown extension")

### Data cleaning

Drop empty rows

In [5]:
# Drop empty rows and columns used for padding in excel
f.dropna(axis=0,how='all',inplace=True)
f.dropna(axis=1,how='all',inplace=True)
f.reset_index(drop=True, inplace = True)

Change column names

In [6]:
# Other columns are treated as DateTime, so store their values as a string and rename the other columns as required
f.columns = [str(x) if '00' not in str(x) else str(x)[:-9] for x in f.loc[0,:].tolist()]
# trackerID is the subcategory
f.rename(columns={'nan':'trackerID'},inplace=True)
# replace a column name if it contains some additional data
l = [x.strip() for x in f.columns.tolist()]
for i in range(len(l)):
    if '(' in l[i]:
        l[i] = '2020-08-21'
f.columns = l

Drop un-necessary columns

In [7]:
# drop the first row
f.drop(0,axis=0,inplace=True)
f.reset_index(drop=True, inplace=True)

In [8]:
# Remove all the rows which do not contain an upper and lower bound
f = f[f[UPPER].notnull() & f[LOWER].notnull()]

In [9]:
f = f.set_index(['section','trackerID']) # same as f.pivot_table(index=['section','Obj'],aggfunc='first')
index_names = f.index.names.copy()

In [10]:
# get the column names except for the upper and lower bounds
col_list = f.columns.tolist()
col_list.remove(UPPER)
col_list.remove(LOWER)

# take the transpose and unstack to get date as index (reset the index to set the column name) 
temp = f[col_list].T.unstack().to_frame().reset_index()
# rename the column, set the index and join with f to get the upper and lower bounds too
g = temp.rename(columns={'level_2':'date', 0:'value'})
g.loc[:, 'date'] = pd.to_datetime(g['date'])

In [11]:
f['UPPER'].replace('[a-zA-Z]*',np.nan,regex=True,inplace=True)
f['LOWER'].replace('[a-zA-Z]*',np.nan,regex=True,inplace=True)
g['value'].replace('[a-zA-Z]*',np.nan,regex=True,inplace=True)

## Interpolate values
Value before interpolation

In [12]:
temp = g.copy()
temp.index = temp['date']
del temp['date']
temp[temp['trackerID'] == 'Haemoglobin'].sort_values('date').tail(20)

,section,trackerID,value
date,,,
2020-07-25,HEMATOLOGY,Haemoglobin,8.2
2020-07-26,HEMATOLOGY,Haemoglobin,NaN
2020-07-27,HEMATOLOGY,Haemoglobin,7.5
2020-07-28,HEMATOLOGY,Haemoglobin,7.6
2020-08-01,HEMATOLOGY,Haemoglobin,7.8
2020-08-04,HEMATOLOGY,Haemoglobin,NaN
2020-08-06,HEMATOLOGY,Haemoglobin,6.8
2020-08-10,HEMATOLOGY,Haemoglobin,8.8
2020-08-13,HEMATOLOGY,Haemoglobin,8.9


In [13]:
interpolated = temp.groupby(['section', 'trackerID'])\
                    .resample('1D')\
                    .mean()
interpolated['interpolated_value'] = interpolated['value'].interpolate()
interpolated = interpolated.reset_index()
interpolated[interpolated['trackerID'] == 'Haemoglobin'].sort_values('date').tail(20)

,section,trackerID,date,value,interpolated_value
1528,HEMATOLOGY,Haemoglobin,2020-11-06,NaN,10.223810
1529,HEMATOLOGY,Haemoglobin,2020-11-07,NaN,10.271429
1530,HEMATOLOGY,Haemoglobin,2020-11-08,NaN,10.319048
1531,HEMATOLOGY,Haemoglobin,2020-11-09,NaN,10.366667
1532,HEMATOLOGY,Haemoglobin,2020-11-10,NaN,10.414286
1533,HEMATOLOGY,Haemoglobin,2020-11-11,NaN,10.461905
1534,HEMATOLOGY,Haemoglobin,2020-11-12,NaN,10.509524
1535,HEMATOLOGY,Haemoglobin,2020-11-13,NaN,10.557143
1536,HEMATOLOGY,Haemoglobin,2020-11-14,NaN,10.604762
1537,HEMATOLOGY,Haemoglobin,2020-11-15,NaN,10.652381


In [14]:
f = interpolated.set_index(index_names+['date']).join(f.loc[:,[UPPER,LOWER]])

In [15]:
# creating a column with normalised value
upper = f[UPPER]
lower = f[LOWER]
f['normalised'] = ((f['value'] - (upper+lower)/2)/((upper-lower)/2)+1)/2

In [16]:
# round all the datapoints
df = f.round(3)

In [17]:
df

value  interpolated_value  UPPER  LOWER  \
section        trackerID date                                                  
BIOCHEMISTRY   Albumin   2020-07-20   4.03                4.03    5.0    3.5   
                         2020-07-21    NaN                3.65    5.0    3.5   
                         2020-07-22   3.27                3.27    5.0    3.5   
                         2020-07-23    NaN                3.28    5.0    3.5   
                         2020-07-24    NaN                3.29    5.0    3.5   
...                                    ...                 ...    ...    ...   
URINE ANALYSIS PTH       2020-11-21    NaN               46.40   68.3   15.0   
                         2020-11-22    NaN               46.40   68.3   15.0   
                         2020-11-23    NaN               46.40   68.3   15.0   
                         2020-11-24    NaN               46.40   68.3   15.0   
                         2020-11-25    NaN               46.40   68.3   15.0   

                                     normalised  
section        trackerID date                    
BIOCHEMISTRY   Albumin   2020-07-20       0.353  
                         2020-07-21         NaN  
                         2020-07-22      -0.153  
                         2020-07-23         NaN  
                         2020-07-24         NaN  
...                                         ...  
URINE ANALYSIS PTH       2020-11-21         NaN  
                         2020-11-22         NaN  
                         2020-11-23         NaN  
                         2020-11-24         NaN  
                         2020-11-25         NaN  

[2967 rows x 5 columns]

In [18]:
df.to_csv('input.csv')

### Store a copy of original, uninterpolated data

In [43]:
# Store a copy of uninterpolated data
temp = g.copy()
temp['interpolated_value'] = temp['value']
df_original = temp.set_index(index_names+['date']).join(f.loc[:,[UPPER,LOWER]])

upper = df_original[UPPER]
lower = df_original[LOWER]

df_original['normalised'] = ((df_original['value'] - (upper+lower)/2)/((upper-lower)/2)+1)/2
df_original = df_original.round(3)

df_original

value  interpolated_value  UPPER  \
section        trackerID   date                                           
HEMATOLOGY     Haemoglobin 2020-11-25    9.9                 9.9   17.0   
                           2020-11-16   10.7                10.7   17.0   
                           2020-10-26    9.7                 9.7   17.0   
                           2020-10-16   11.4                11.4   17.0   
                           2020-09-25    9.4                 9.4   17.0   
...                                      ...                 ...    ...   
URINE ANALYSIS PTH         2020-07-24    NaN                 NaN   68.3   
                           2020-07-23    NaN                 NaN   68.3   
                           2020-07-22    NaN                 NaN   68.3   
                           2020-07-21    NaN                 NaN   68.3   
                           2020-07-20    NaN                 NaN   68.3   

                                       LOWER  normalised  
section        trackerID   date                           
HEMATOLOGY     Haemoglobin 2020-11-25   13.0      -0.775  
                           2020-11-16   13.0      -0.575  
                           2020-10-26   13.0      -0.825  
                           2020-10-16   13.0      -0.400  
                           2020-09-25   13.0      -0.900  
...                                      ...         ...  
URINE ANALYSIS PTH         2020-07-24   15.0         NaN  
                           2020-07-23   15.0         NaN  
                           2020-07-22   15.0         NaN  
                           2020-07-21   15.0         NaN  
                           2020-07-20   15.0         NaN  

[575 rows x 5 columns]